In [ ]:
import numpy as np
import cv2 as cv
import sys

# [1] YOLOv3 모델과 클래스 이름 구성 함수
def construct_yolo_v3():
    # coco_names.txt에서 COCO 데이터셋의 클래스 이름 불러오기
    f = open('coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]  # 각 줄의 개행문자 제거
    
    # YOLOv3 가중치(.weights)와 설정 파일(.cfg) 불러오기
    model = cv.dnn.readNet('yolov3.weights', 'yolov3.cfg')
    
    # 모델의 모든 레이어 이름 가져오기
    layer_names = model.getLayerNames()
    
    # 출력층만 선택 (탐지 결과가 나오는 레이어)
    out_layers = [layer_names[i - 1] for i in model.getUnconnectedOutLayers()]
    
    return model, out_layers, class_names

# [2] YOLO로 이미지에서 객체 탐지하는 함수
def yolo_detect(img, yolo_model, out_layers):
    # 입력 이미지 크기 가져오기
    height, width = img.shape[0], img.shape[1]

    # 입력 이미지 blob으로 전처리
    # scale factor=1/256로 정규화, 크기 448x448 리사이즈, BGR → RGB 변환
    test_img = cv.dnn.blobFromImage(
        img,
        1.0/256,
        (448, 448),
        (0, 0, 0),
        swapRB=True
    )

    # 모델에 blob 입력
    yolo_model.setInput(test_img)

    # forward() 호출하여 출력 레이어 실행 → 탐지 결과
    output3 = yolo_model.forward(out_layers)

    box, conf, id = [], [], []  # 바운딩박스, 신뢰도, 클래스ID 저장

    # 출력 레이어별 탐지 결과 처리
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]  # 앞 5개는 박스 좌표, 이후는 클래스별 점수
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # 신뢰도가 0.5 이상인 객체만 사용
            if confidence > 0.5:
                centerx, centery = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(centerx - w/2), int(centery - h/2)
                box.append([x, y, x+w, y+h])   # [x1, y1, x2, y2]
                conf.append(float(confidence))
                id.append(class_id)

    # Non-Maximum Suppression으로 겹치는 박스 제거
    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)

    # 최종 탐지 객체 리스트: [좌표 4개 + 신뢰도 + 클래스ID]
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]

    return objects

# [3] YOLOv3 모델과 클래스 불러오기
model, out_layers, class_names = construct_yolo_v3()

# [4] 클래스별 랜덤 색상 생성 (바운딩박스 컬러)
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

# [5] 비디오 캡처 장치 열기 (웹캠)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)  # CAP_DSHOW는 Windows에서 DirectShow 사용
if not cap.isOpened():
    sys.exit('카메라 연결 실패')

# [6] 프레임 반복 처리
while True:
    ret, frame = cap.read()  # 프레임 읽기
    if not ret:
        sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
    
    # YOLO로 객체 탐지 수행
    res = yolo_detect(frame, model, out_layers)
 
    # 탐지된 객체 시각화
    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = str(class_names[id]) + ' %.3f' % confidence
        cv.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)  # 박스 그리기
        cv.putText(frame, text, (x1, y1 + 30), cv.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)

    # 결과 프레임 표시
    cv.imshow("Object detection from video by YOLO v.3", frame)
    
    key = cv.waitKey(1)  # 1ms 대기: 'q' 입력시 종료
    if key == ord('q'):
        break

# [7] 리소스 정리
cap.release()          # 카메라 장치 닫기
cv.destroyAllWindows() # 모든 OpenCV 윈도우 닫기
